<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Q&A Classification Evals</h1>

The purpose of this notebook is:

- to evaluate the performance of an LLM-assisted approach to detecting issues with Q&A systems on retrieved context data
- to provide an experimental framework for users to iterate and improve on the default classification template.

## Install Dependencies and Import Libraries

In [ ]:
#####################
## N_EVAL_SAMPLE_SIZE
#####################
# Eval sample size determines the run time
# 100 samples: GPT-4 ~ 80 sec / GPT-3.5 ~ 40 sec
# 1,000 samples: GPT-4 ~15-17 min / GPT-3.5 ~ 6-7min (depending on retries)
# 10,000 samples GPT-4 ~170 min / GPT-3.5 ~ 70min
N_EVAL_SAMPLE_SIZE = 100

In [ ]:
!pip install -qq "arize-phoenix[experimental]" ipython matplotlib openai pycm scikit-learn tiktoken

In [ ]:
import os
from getpass import getpass

import matplotlib.pyplot as plt
import openai
import pandas as pd

In [ ]:
import phoenix.experimental.evals.templates.default_templates as templates
from phoenix.experimental.evals import (
    OpenAIModel,
    download_benchmark_dataset,
    llm_eval_binary,
)
from pycm import ConfusionMatrix
from sklearn.metrics import classification_report

pd.set_option("display.max_colwidth", None)

## Download Benchmark Dataset



- Squad 2:
The 2.0 version of the large-scale dataset Stanford Question Answering Dataset (SQuAD 2.0) allows researchers to design AI models for reading comprehension tasks under challenging constraints.
https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/default/15785042.pdf
- Supplemental Data to Sqaud 2: In order to check the case of detecting incorrect answers, we created wrong answers based on the context data. The wrong answers are intermixed with right answers.
- sampled_answer is a sampled column of randomly original Squad 2 or incorrect answers

In [ ]:
df = download_benchmark_dataset(task="qa-classification", dataset_name="qa_generated_dataset")
df.head()

- **question**: This is the question the Q&A system is running against
- **sampled_answer**: This is a random sample of correct_answer from Squad 2 or wrong_answer which is a made up incorrect answer. THIS IS the column we test against as it has wrong and wright answers.
- **correct_answer**: True if answer is correct, False if not. The ground truth to test against.
- **answer**: This is the answer to the right answer to the question.
- **wrong_answer**: This is an incorrect answer generated by the context.
- **context**: This is the context to be used to answer the question, and is what Q&A Eval must use to check the correct answer.



In [ ]:
df.head()

## Display Binary Q&A Classification Template

View the default template used to classify hallucinations. You can tweak this template and evaluate its performance relative to the default.

In [ ]:
print(templates.QA_PROMPT_TEMPLATE_STR)

## Configure the API Key

Configure your OpenAI API key.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## Benchmark Dataset Sample
Sample size determines run time
Recommend iterating small: 100 samples
Then increasing to large test set

In [ ]:
df_sample = df.sample(n=N_EVAL_SAMPLE_SIZE).reset_index(drop=True)

## LLM Evals: Q&A Classifications GPT-4
Run Q&A classifications against a subset of the data.

Instantiate the LLM and set parameters.

In [ ]:
model = OpenAIModel(
    model_name="gpt-4",
    temperature=0.0,
)

In [ ]:
model("Hello world, this is a test if you are working?")

Run LLM Eval using the template against the dataset: This is the main Eval function

In [ ]:
# The rails fore the output to specific values of the template
# It will remove text such as ",,," or "...", anything not the
# binary value expected from the template
rails = list(templates.QA_PROMPT_RAILS_MAP.values())
Q_and_A_classifications = llm_eval_binary(
    dataframe=df_sample,
    template=templates.QA_PROMPT_TEMPLATE_STR,
    model=model,
    rails=rails,
)



Evaluate the predictions against human-labeled ground-truth Q&A labels.

In [ ]:
true_labels = df_sample["answer_true"].map(templates.QA_PROMPT_RAILS_MAP).tolist()
Q_and_A_classifications = (
    pd.Series(Q_and_A_classifications).map(lambda x: "unparseable" if x is None else x).tolist()
)

print(classification_report(true_labels, Q_and_A_classifications, labels=rails))
confusion_matrix = ConfusionMatrix(
    actual_vector=true_labels, predict_vector=Q_and_A_classifications, classes=rails
)
confusion_matrix.plot(
    cmap=plt.colormaps["Blues"],
    number_label=True,
    normalized=True,
);

##  LLM Evals: Q&A Classifications GPT-3.5


Evaluate the predictions against human-labeled ground-truth Q&A labels.

In [ ]:
model = OpenAIModel(model_name="gpt-3.5-turbo", temperature=0.0, request_timeout=20)

In [ ]:
Q_and_A_classifications = llm_eval_binary(
    dataframe=df_sample,
    template=templates.QA_PROMPT_TEMPLATE_STR,
    model=model,
    rails=templates.QA_PROMPT_RAILS_MAP.values(),
)

In [ ]:
true_labels = df_sample["answer_true"].map(templates.QA_PROMPT_RAILS_MAP).tolist()
classes = list(templates.QA_PROMPT_RAILS_MAP.values())
Q_and_A_classifications = (
    pd.Series(Q_and_A_classifications).map(lambda x: "unparseable" if x is None else x).tolist()
)

print(classification_report(true_labels, Q_and_A_classifications, labels=classes))
confusion_matrix = ConfusionMatrix(
    actual_vector=true_labels, predict_vector=Q_and_A_classifications, classes=classes
)
confusion_matrix.plot(
    cmap=plt.colormaps["Blues"],
    number_label=True,
    normalized=True,
);